In [1]:
# Code to analyse EM comparisons. The data must be input via excel spreadsheets filled out with the information about the 
# comparison in the first sheet, and the measurements, uncertainties and any correlations in the second sheet. It is important
# that this is correctly filled out, otherwise the code won't work. There is a template showing how to do this in the folder
# on the I: drive.

# This uses code written by Annette Koo for CCPR comparisons, edited to be used in EM comparisons. It assumes that the KCRV is
# directly calculated as the weighted mean of the (linked) participants, and uses generalised least squares to calculate this.
# If the analysis is done using other methods, this will need to be edited in order to perform the analysis.

# 11 March 2019
# Edited 9 May to tidy up and add comments.

In [2]:
import wx
import wx.lib.agw.aui as aui
import wx.lib.scrolledpanel as scrolled
import wx.grid
import os
import sys, os.path
from textwrap import wrap
import copy
import numpy as np
import numpy.linalg as linalg
import xlrd
import xlwt
import urllib

import math
from scipy import stats
import numpy as np
import numpy.linalg as linalg
import pandas
from ipy_table import *
from xlutils.copy import copy
import matplotlib.gridspec as gridspec
from matplotlib import rc, font_manager
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display_png
from datetime import *
import pdfkit
from scipy.stats import chi2
import operator as op
import itertools

from matplotlib.backends.backend_wxagg import FigureCanvasWxAgg as FigureCanvas
from matplotlib.backends.backend_wxagg import NavigationToolbar2WxAgg
from matplotlib.figure import Figure
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rc
from matplotlib.mathtext import MathTextParser
mathtext_parser = MathTextParser("Bitmap")

In [3]:
def read_data(filename):
    # Get information about comparison (Number of artefacts (F), participants (P), measurements (N))
    book = xlrd.open_workbook(filename)
    sheet1 = book.sheet_by_index(0)
    
    # Type of comparison
    TypeComp = str(sheet1.cell_value(1,1))
    
    if TypeComp != "Primary" and TypeComp != "Linked":
        raise ValueError("Comparison Type must be either Primary or Linked")

    # Save the number of artefacts, participants and measurements
    try:
        NumArtefacts = int(sheet1.cell_value(2,1))
        NumParticipants = int(sheet1.cell_value(3,1))
        NumMeasurements = int(sheet1.cell_value(4,1))
    except:
        raise ValueError("The spreadsheet has not been correctly filled in.")
    
    if TypeComp == "Linked":
        try:
            NumLinking = int(sheet1.cell_value(5,1))
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    else:
        NumLinking = 0

    # Save artefact and participant names
    ArtefactNames = []
    for i in range(0,NumArtefacts):
        try:
            ArtefactNames.append(sheet1.cell_value(8,i+1))
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")

    ParticipantNames = []
    linking = np.zeros((NumMeasurements,1))
    for i in range(0,NumParticipants):
        try:
            ParticipantNames.append(sheet1.cell_value(i+9,0))
            linking[i,0] = sheet1.cell_value(i+9,2)
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    
    # Get the measurements (y) and covariance matrix (U) from the comparison, and the linking data. Save participant names in P
    sheet2 = book.sheet_by_index(1)
    y = np.zeros((NumMeasurements+NumLinking,1))
    U = np.zeros((NumMeasurements+NumLinking, NumMeasurements+NumLinking))
    P = []
    for i in range(0,NumMeasurements+NumLinking):
        try:
            y[i,0] = sheet2.cell_value(i+3,1)
            P.append(sheet2.cell_value(i+3,2)) 
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    
        for j in range(0,NumMeasurements+NumLinking):
            try:
                U[i,j] = sheet2.cell_value(i+3,j+7)
            except:
                raise ValueError("The spreadsheet has not been correctly filled in.")

    # Save the measurement titles
    MeasurementTitles = []
    for i in range(0,NumMeasurements+NumLinking):
        try:
            MeasurementTitles.append([sheet2.cell_value(i+3,2)])
            MeasurementTitles[i].append(sheet2.cell_value(i+3,3))
            MeasurementTitles[i].append(sheet2.cell_value(i+3,4))
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    
    
    # Create the design matrix X, and calculate the weights.
    # Initialise matrices to store X
    if TypeComp == "Primary":
        X = np.zeros((NumMeasurements, NumArtefacts+NumParticipants))
    elif TypeComp == "Linked":
        X = np.zeros((NumMeasurements+NumLinking, NumArtefacts+NumParticipants))
        

    # Create vectors to store the average uncertainty for each NMI (used to determine the weights)
    ave_U = np.zeros(NumParticipants)
    count_per_lab = np.zeros(NumParticipants)
    
    # Initialise w to store the weights. The weight vector has length F+P, with the first F rows of w all 0's
    w = np.zeros((NumArtefacts+NumParticipants, 1))

    # Fill in 1's in the appropriate places in X
    
    if TypeComp == "Primary":
        # For primary comparison
        try:
            for i in range(0, NumMeasurements):
            # Iterate through the number of measurements

                for j in range(0, NumArtefacts):
                    # Iterate through the number of artefacts
                    if MeasurementTitles[i][1] == ArtefactNames[j]:
                        X[i,j] = 1.0

                for k in range(0, NumParticipants):
                    # Iterate through the number of participants
                    if MeasurementTitles[i][0] == ParticipantNames[k]:
                        X[i,NumArtefacts+k] = 1.0

                        # Update the uncertainty squared sum for the NMI
                        ave_U[k] += np.sqrt(U[i,i])
                        count_per_lab[k] += 1

            # Determine the average uncertainty for each NMI
            ave_U = ave_U / count_per_lab

            # Update the non-zero entries of w
            for i in range(0, NumParticipants):
                w[NumArtefacts+i, 0] = 1/(ave_U[i]**2)

            # Standardise the weights so they sum to 1
            w = w/sum(w)
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    
    
    else:      
        # For linked comparison
        try:
            for i in range(0, NumMeasurements+NumLinking):
                # Iterate through the number of measurements

                for j in range(0, NumArtefacts):
                    # Iterate through the number of artefacts
                    if MeasurementTitles[i][1] == ArtefactNames[j]:
                        X[i,j] = 1.0

                for k in range(0, NumParticipants):
                    # Iterate through the number of participants

                    if MeasurementTitles[i][0] == ParticipantNames[k]:
                        X[i,NumArtefacts+k] = 1.0

                    elif MeasurementTitles[i][0] == ParticipantNames[k] + " link":
                        X[i,NumArtefacts+k] = 1.0
        except:
            raise ValueError("The spreadsheet has not been correctly filled in.")
    
    return(y, U, w, X, ave_U, NumArtefacts,NumMeasurements,NumParticipants,ParticipantNames,MeasurementTitles,count_per_lab,P,
          NumLinking,ArtefactNames)

In [4]:
def calculate_beta(y, U, X, w):
    # Calculates gamma, beta, CovBeta, ChiSq using GLS
    # Takes in: measurements (y), covariance matrix (U), design matrix (X), weights (w)
    # Returns:  gamma, beta, CovBeta
    
    gamma = np.linalg.multi_dot([ linalg.inv( np.linalg.multi_dot([X.T,linalg.inv(U),X])), X.T, linalg.inv(U)])
    beta = np.dot(gamma,y)
    CovBeta = np.linalg.multi_dot([ gamma, U, gamma.T])
    
    return(gamma, beta, CovBeta)

In [5]:
def model_B(y, U, X, beta):
    # Model B - calculates ChiSq using Model B
    # y = theta + delta + e
    # Takes in: measurements (y), covariance matrix (U), design matrix (X), beta
    # Returns:  ChiSq calculated using Model B
    
    # Calculate ChiSq
    ChiSq_B = np.linalg.multi_dot([ (y - np.dot(X, beta)).T, linalg.inv(U), (y - np.dot(X, beta))])
    
    return(ChiSq_B)

In [6]:
def check_analysis(file, directory):
    
    # Read data in from file
    [y, U, w, X, ave_U, NumArtefacts, NumMeasurements, NumParticipants, ParticipantNames, MeasurementTitles, count_per_lab, P,
          NumLinking, ArtefactNames] = read_data(file)

    # Calculate gamma, beta, CovBeta
    [gamma, beta, CovBeta] = calculate_beta(y,U,X,w)

    # Create workbook to save data
    wb = xlwt.Workbook()

    # Add sheet with measurements
    ws = wb.add_sheet('y')
    for i in range(0,NumMeasurements):
        ws.write(i,0,y[i,0])
        ws.write(i,1,P[i])

    # Add sheet with design matrix, X
    ws = wb.add_sheet('X')
    for i in range(0,NumMeasurements):
        for j in range(0,NumArtefacts+NumParticipants):
            ws.write(i+1,j+1,X[i,j])

    # Add sheet with the uncertainties used in the weights calculations
    ws = wb.add_sheet('U for weights')
    ws.write(0,0,'Original avg U')
    ws.write(0,1,'Weights')
    ws.write(0,2,'Participant')
    for i in range(0,NumParticipants):
        ws.write(i+1,0,ave_U[i])
        ws.write(i+1,1,w[i+NumArtefacts,0])
        ws.write(i+1,2,ParticipantNames[i])

    # Add sheet with the original covariance matrix
    ws = wb.add_sheet('Umatrix')
    for i in range(0,NumMeasurements):
        for j in range(0,NumMeasurements):
            ws.write(i,j,U[i,j])

    # Add sheet with beta before applying Mandel-Paule
    ws = wb.add_sheet('Beta')
    for i in range(0,NumParticipants+NumArtefacts):
        ws.write(i,0,beta[i,0])
        if i>(NumArtefacts-1):
            ws.write(i,1,ParticipantNames[i-NumArtefacts])

    # Add sheet with the covariance matrix of beta
    ws = wb.add_sheet('CovBeta')
    for i in range(0,NumParticipants):
        ws.write(i,0,np.sqrt(CovBeta[i,i]))
        for j in range(0,NumParticipants):
            ws.write(i,j+2,CovBeta[i,j])   
    
    # Add sheet with the KCRVs
    ws = wb.add_sheet('KCRVs')
    ws.write(0,0, 'Artefact')
    ws.write(0,1, 'KCRV')
    for i in range(0,NumArtefacts):
        ws.write(i+1,0,ArtefactNames[i])
        ws.write(i+1,1,beta[i,0])

    # Add sheet with the degrees of equivalence
    ws = wb.add_sheet('DoEs')
    ws.write(0,0,'Participant')
    ws.write(0,1,'Degree of Equivalence')
    for i in range(0,NumParticipants):
        ws.write(i+1,0,ParticipantNames[i])
        ws.write(i+1,1,beta[i+NumArtefacts,0])

    # Add sheet with the uncertainties in the degrees of equivalence
    ws = wb.add_sheet('U(DoEs)')
    ws.write(0,0,'Participant')
    ws.write(0,1,'Uncertainty in Degree of Equivalence')
    for i in range(0,NumParticipants):
        ws.write(i+1,0,ParticipantNames[i])
        ws.write(i+1,1,np.sqrt(CovBeta[i+NumArtefacts,i+NumArtefacts]))
    
    

    wb.save(file+' checked.xls')
    
    return(wb)

In [8]:
# Set up some button numbers for the menu

ID_ABOUT=101
ID_OPEN=102
ID_SAVE=103
ID_BUTTON1=300
ID_EXIT=200

def gif(eq):
    page = "http://www.forkosh.com/mimetex.cgi?"+eq
    image = urllib.URLopener()
    image.retrieve(page,repr(os.path.dirname(os.path.realpath(sys.argv[0])))[1:-1]+'\\eqn.gif')

def mathtext_to_wxbitmap(eq):
    ftimage, depth = mathtext_parser.parse(eq, 150)
    return wx.BitmapFromBufferRGBA(
        ftimage.get_width(), ftimage.get_height(),
        ftimage.as_rgba_str())

In [9]:
class PageZero(scrolled.ScrolledPanel):
    # Set up front page, where user inputs the type of comparison and number of artefacts and participants
    # Allows user to change the number and to give the artefacts and partipants names
    
        def __init__(self, parent):
            scrolled.ScrolledPanel.__init__(self, parent)
            
            self.Data = ['File', 'Directory', 'Calculate Results']

            # Request comparison information
            wx.StaticBox(self,-1,"",wx.Point(15,15),wx.Size(800,85))
            
            # Select file to check analysis for
            self.lblFile = wx.StaticText(self, -1, "Select file of comparison", wx.Point(20,30))
            self.editFile = wx.FilePickerCtrl(self, 20, self.Data[0], pos=wx.Point(200, 30), size=wx.Size(270,-1))
            wx.EVT_FILEPICKER_CHANGED(self,20,self.OnClick)
            
            # Select directory to save results
            self.lblDir = wx.StaticText(self, -1, "Select folder to save results", wx.Point(20,55))
            self.editDir = wx.DirPickerCtrl(self, 23, self.Data[1], pos=wx.Point(200, 55), size=wx.Size(270,-1))
            wx.EVT_DIRPICKER_CHANGED(self,23,self.OnClick)
            
            # Create 'Go' button to tell app to do analysis and tell user that it's done
            self.startAnalysis = wx.Button(self, 26, self.Data[2], pos=wx.Point(200, 80), size=wx.Size(270,-1))
            wx.EVT_BUTTON(self,26,self.OnToggle)
            self.startAnalysis.Bind(wx.EVT_BUTTON, self.OnToggle) 

            self.h = 205
            self.NAClicks = 0
            self.NPClicks = 0
            
            
        def OnClick(self,event):
            # Event for selecting file and directory
            if event.GetId() == 20:
                self.Data[0] = self.editFile.GetPath()  
            
            if event.GetId() == 23:
                self.Data[1] = self.editDir.GetPath()
                
                    
        def OnToggle(self,event): 
            # Event for selecting 'Go'
            wb = check_analysis(self.Data[0],self.Data[1])
            self.finished = wx.StaticText(self, -1, "The results can be found in the directory you selected.", 
                                                  wx.Point(20,105))                   

In [10]:
class MainWindow(wx.Frame):
    # Set up main window
    
    def __init__(self,parent,title):
        # Based on a frame, so set up the frame
        wx.Frame.__init__(self,parent,wx.ID_ANY, title)

        self.CreateStatusBar() # A Statusbar in the bottom of the window
        p = wx.Panel(self)
        bookstyle = aui.AUI_NB_TOP
        bookstyle&=~(aui.AUI_NB_SCROLL_BUTTONS)
        self.nb = aui.AuiNotebook(p,style=bookstyle)
        self.NTClicks=0
        
        # Page titles
        page0=PageZero(self.nb) # set up page 0
        self.nb.AddPage(page0, "Comparison Information")
        
        # Setting up the menus. 
        self.filemenu = wx.Menu()
        self.calcmenu = wx.Menu()
        self.helpmenu = wx.Menu()

        # The & character indicates the short cut key
        self.filemenu.Append(ID_SAVE, "&Save"," Save file")
        self.filemenu.AppendSeparator()
        self.helpmenu.Append(ID_ABOUT, "&About"," Information about this program")
        self.filemenu.AppendSeparator()
        self.filemenu.Append(ID_EXIT,"E&xit"," Terminate the program")

        # Creating the menubar.
        menuBar = wx.MenuBar()
        menuBar.Append(self.filemenu,"&File") # Adding the "filemenu" to the MenuBar
        #menuBar.Append(self.calcmenu,"&Calculate") # Adding the "calcmenu" to the MenuBar
        menuBar.Append(self.helpmenu,"&Help") # Adding the "helpmenu" to the MenuBar
        self.SetMenuBar(menuBar) # Adding the MenuBar to the Frame content.

        # Define the code to be run when a menu option is selected
        wx.EVT_MENU(self, ID_ABOUT, self.OnAbout)
        wx.EVT_MENU(self, ID_EXIT, self.OnExit)
        
        self.sizer=wx.BoxSizer()
        self.sizer.Add(self.nb,1,wx.EXPAND)
        self.x = 850
        self.y = 600
        self.SetClientSize(wx.Size(self.x, self.y)) 

        # Tell it which sizer is to be used for main frame
        # It may lay out automatically and be altered to fit window
        p.SetAutoLayout(1)
        p.SetSizer(self.sizer)
        p.Fit()
        
        # Show it !!!
        self.Show(1)

        # Define widgets early even if they're not going to be seen so that they can come up FAST when someone clicks for them!
        self.aboutme = wx.MessageDialog( self, " This tool for comparison analysis \n"
                                               " was created to make use of GLS accessible. \n"
                                               " Annette Koo, January 2012",
                            "About Comparison Analysis Tool", wx.OK)
        self.doiexit = wx.MessageDialog( self, " Exit - Are You Sure? \n",
                        "Exiting", wx.YES_NO)
        self.dirname = ''
        
    def OnAbout(self,e):
        # A modal to describe comparison analysis tool
        self.aboutme.ShowModal() # Shows it
           
    def OnExit(self,e):
        # A modal with an "are you sure" check
        igot = self.doiexit.ShowModal() # Shows it
        if igot == wx.ID_YES:
            self.Close(True)

In [11]:
app = wx.App()
view = MainWindow(None, "Comparison Analysis")

# Enter event loop
app.MainLoop()